In [313]:
import json

import numpy as np
from scipy.spatial import KDTree
from scipy.optimize import linear_sum_assignment

In [735]:
#np.random.seed(42)

def random_spots_in_radius(n_spots, n_dim, radius):
    """
    get random relative (integer) coordinates within radius
    """
    
    res_spots = []
    while len(res_spots) < n_spots:
        # uniformly distributed on hypersquare
        candidate = [np.random.randint(-radius,radius+1) for _ in range(n_dim)]
        # reject spots not in hypersphere
        if (np.sum(np.array(candidate)**2) <= radius**2):
            res_spots.append(candidate)

    return np.array(res_spots)

radius = 100
n_dim = 3
n_spots = 5*16
spots = random_spots_in_radius(n_spots, n_dim, radius)

def get_descriptor(spot, spots, neighbors_for_descriptor = 3):
    kd = KDTree(spots)
    ds, idxs = kd.query(spot, k=neighbors_for_descriptor+1)
    d_vecs = spot - spots[idxs[1:]]
    desc = d_vecs.flatten()
    return desc

def match_kd(descs_a, descs_b):
    kd = KDTree(descs_a)
    ds, idxes = kd.query(descs_b)
    return idxes

def match_la(descs_a, descs_b):
    n_spots = np.array(descs_a).shape[0]
    a = np.tile(descs_a, [n_spots, 1] )
    b = np.repeat(descs_b, n_spots, 0)
    ds = np.sqrt(np.sum((a-b)**2, axis=1)).reshape((n_spots, n_spots))
    _, idxes_la = linear_sum_assignment(ds)
    return idxes_la

descs_gt = [get_descriptor(s, spots) for s in spots]
perm = np.arange(n_spots)
np.random.shuffle(perm)
spots_t = spots[perm]
spots_t += np.array([210] * n_dim)
descs_shuff = [get_descriptor(s, spots_t) for s in spots_t]


idxes = match_kd(descs_gt, descs_shuff)
idxes_la = match_la(descs_gt, descs_shuff)

#print(perm)
#print(idxes)
#print(idxes_la)

(np.sum(idxes_la!=perm), np.sum(perm!=idxes))

[33 75 35  9 50 71 68 22 59  7 62 44  3 29 18 48 13 15 43 17  5 40 20 11
 24 10 31 23 56  8 65 78 34  0  6 76 30 14 38 72 19 64 49 61 52 25 74 32
 51 26 39  4 67 37 41 45 70 42 77 53 79 16 54 60  1 46 58 73  2 66 69 12
 36 63 47 28 27 57 55 21]
[33 75 35  9 50 71 68 22 59  7 62 44  3 29 18 48 13 15 43 17  5 40 20 11
 24 10 31 23 56  8 65 78 34  0  6 76 30 14 38 72 19 64 49 61 52 25 74 32
 51 26 39  4 67 37 41 45 70 42 77 53 79 16 54 60  1 46 58 73  2 66 69 12
 36 63 47 28 27 57 55 21]
[33 75 35  9 50 71 68 22 59  7 62 44  3 29 18 48 13 15 43 17  5 40 20 11
 24 10 31 23 56  8 65 78 34  0  6 76 30 14 38 72 19 64 49 61 52 25 74 32
 51 26 39  4 67 37 41 45 70 42 77 53 79 16 54 60  1 46 58 73  2 66 69 12
 36 63 47 28 27 57 55 21]


(0, 0)